In [184]:
import numpy as np

from model.net import *
from data_prep.data_loader import * 

import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### layers

Structure of our model: `Embedding` --> `LSTM` --> `Dropout`--> `LSTM` --> `Dropout` --> `FC`. Other core features of our model:

- we are going to use pretrained embeddings;
- we need only last output of the 2nd layer of `LSTM`;
- we experiment with different `seq_len`, we added this parameter to `params.json` file (in addition to standard params of cs230);
- we also experiment with dropout (and we also added this parameter to `params.json`);

In [185]:
dataset_params = utils.Params('data/5w/dataset_params.json')
training_params = utils.Params('experiments/base_model/training_params.json')
rnn = ReviewRNN(dataset_params, training_params)

In [186]:
rnn

ReviewRNN(
  (embedding): Embedding(2697, 50)
  (lstm): LSTM(50, 32, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

### forward pass

In [187]:
torch.manual_seed(42)
data_dir = 'data/5w'
data_params = utils.Params('data/5w/dataset_params.json')
training_params = utils.Params('experiments/base_model/training_params.json')
types = ['train']

dl = DataLoader(data_dir, data_params)
data = dl.load_data(types, data_dir)

it = dl.data_iterator(data['train'], training_params)

In [188]:
x, y = next(it)
x.shape

torch.Size([64, 10])

In [189]:
x[0, :]

tensor([   0,    1,    2,    3,    4, 2695, 2695, 2695, 2695, 2695])

In [190]:
rnn = ReviewRNN(data_params, training_params)

In [191]:
rnn

ReviewRNN(
  (embedding): Embedding(2697, 50)
  (lstm): LSTM(50, 32, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

In [192]:
batch_size = x.shape[0]
batch_size

64

In [193]:
x_embed = rnn.embedding(x) 
x_embed.shape

torch.Size([64, 10, 50])

In [194]:
x_lstm, _ = rnn.lstm(x_embed)
x_lstm.shape

torch.Size([64, 10, 32])

In [195]:
hidden_dim = x_lstm.shape[2]
hidden_dim

32

In [196]:
x_flat = x_lstm.contiguous().view(-1, hidden_dim)
x_flat.shape

torch.Size([640, 32])

In [197]:
x_out = rnn.fc(x_flat)
x_out.shape

torch.Size([640, 2])

In [198]:
output_size = x_out.shape[1]
output_size

2

In [199]:
x_out = x_out.view(batch_size, -1, output_size)
x_out.shape

torch.Size([64, 10, 2])

In [200]:
x_out[0, -1, :]

tensor([0.0280, 0.0760], grad_fn=<SliceBackward>)

In [201]:
x_out = x_out[:, -1, :]

In [202]:
x_out.shape

torch.Size([64, 2])

In [203]:
x_out[0, :]

tensor([0.0280, 0.0760], grad_fn=<SliceBackward>)

In [204]:
# let's compare this with our forward() function
# for this we set dropout = 0
x[0, :]

tensor([   0,    1,    2,    3,    4, 2695, 2695, 2695, 2695, 2695])

In [205]:
x_out_model = rnn.forward(x)

In [206]:
x_out_model.shape

torch.Size([64, 2])

In [207]:
x_out_model[0, :]

tensor([-0.0010,  0.0303], grad_fn=<SliceBackward>)

### accuracy

In [208]:
x_out.shape

torch.Size([64, 2])

In [209]:
x_out_probs = torch.softmax(x_out, dim=1)

In [210]:
x_out[0, :]

tensor([0.0280, 0.0760], grad_fn=<SliceBackward>)

In [211]:
x_out_probs[0, :]

tensor([0.4880, 0.5120], grad_fn=<SliceBackward>)

In [212]:
# let's check that the result is correct
np.exp(0.0187) / (np.exp(0.0187) + np.exp(0.0527))

0.4915008187386872

In [213]:
y_out = np.argmax(x_out_probs.detach().numpy(), axis=1)
y_out.shape

(64,)

In [214]:
y.shape

torch.Size([64, 2])

In [215]:
y_flat = np.argmax(y.detach().numpy(), axis=1)

In [216]:
y_flat.shape

(64,)

In [220]:
comparison = y_out == y_flat

In [224]:
for i, j, k in zip (y_out, y_flat, comparison):
    if (i == j) != k:
        print('some mistakes')
print('okay')

okay


In [225]:
accuracy = np.sum(comparison) / x_out.shape[0]

In [226]:
accuracy

0.890625